# Introduction to EvoXBench

In this notebook, we will demonstrate 
- how to install EvoXBench
- the basics of EvoXBench

**[EvoXBench](https://arxiv.org/abs/2208.04321)** is an efficient platform 
for facilitating neural architecture search (NAS) 
without the requirement of *GPUs* or 
sophisticated deep learning packages, such as *PyTorch, TensorFlow*, etc.

![](https://raw.githubusercontent.com/EMI-Group/evoxbench/main/assets/evoxbench_overview.png)

## 1. Preparation 
Let's perform the following steps to have EvoXBench properly installed. 

First, download the following two files:
- ``database_xxx.zip`` from [Google Drive](https://drive.google.com/file/d/11bQ1paHEWHDnnTPtxs2OyVY_Re-38DiO/view?usp=sharing) or [Baidu NetDisk](https://pan.baidu.com/s/1PwWloA543-81O-GFkA7GKg)
- ``data_xxx.zip`` from [Google Drive](https://drive.google.com/file/d/1fUZtpTjfEQao2unLKaspL8fOq4xdSXt2/view?usp=sharing) or [Baidu NetDisk](https://pan.baidu.com/s/1yopkISKyjbWIHXFV_Op3pg)

Second, unzip these two files and find their paths
- my ``database`` and ``data`` are unzipped to:
```python
    # /Users/luzhicha/Dropbox/2023/github/evoxbench/
    # └─ database/
    # |  |  __init__.py
    # |  |  db.sqlite3
    # |  |  ...
    # |  
    # └─ data/
    #    └─ darts/
    #    └─ mnv3/
    #    └─ ...
```



In [1]:
print('Configurating EvoXBench...')
from evoxbench.database.init import config
# make sure you update these two paths accordingly, and the first path should be for database file
config("/opt/data/private/BigFiles/EvoXBench_tutorial_materials_IEEE CEC’2023 Competition on Multiobjective Neural Architecture Search/database",
       "/opt/data/private/BigFiles/EvoXBench_tutorial_materials_IEEE CEC’2023 Competition on Multiobjective Neural Architecture Search/data")

Configurating EvoXBench...
Auto Configuration Succeed!, Using database /opt/data/private/BigFiles/EvoXBench_tutorial_materials_IEEE CEC’2023 Competition on Multiobjective Neural Architecture Search/database.
Configuration Succeed!


Good! Now we have successfully installed and configured **EvoXBench**. Let's now get started with some quick examples.

# 2.1 How to create a NAS benchmark (search space)

**EvoXBench** currently supports the following seven search spaces

| $\Omega$ | $D$ | $|\Omega|$ | Objectives | Dataset |
|:-:|:-:|:-:|:-:|:-:|
| [NB101](https://github.com/google-research/nasbench) | 26 |423K | $f^{e}$, ${f}^{c}$ | CIFAR-10 |
| [NB201](https://github.com/D-X-Y/NAS-Bench-201) | 6 | 15.6K | $f^{e}$, ${f}^{c}$, ${f}^{\mathcal{H}}$ | CIFAR-10 |
| [NATS](https://github.com/D-X-Y/NATS-Bench) | 5 | 32.8K | $f^{e}$, ${f}^{c}$, ${f}^{\mathcal{H}}$ | CIFAR-10 |
| [DARTS](https://github.com/automl/nasbench301) | 32 | $\sim10^{21}$ | $f^{e}$, ${f}^{c}$ | CIFAR-10 |
| [ResNet-50](https://github.com/mit-han-lab/once-for-all) | 25 | $\sim10^{14}$ | $f^{e}$, ${f}^{c}$ | ImageNet-1K |
| [Transformer](https://github.com/microsoft/Cream/tree/main/AutoFormer) | 34 | $\sim10^{14}$ | $f^{e}$, ${f}^{c}$ | ImageNet-1K |
| [MNV3](https://github.com/mit-han-lab/once-for-all) | 21 | $\sim10^{20}$ | $f^{e}$, ${f}^{c}$, ${f}^{\mathcal{H}}$ | ImageNet-1K |


In [4]:
# NAS-Bench-101 search space
from evoxbench.benchmarks import NASBench101Benchmark
objs = 'err&params&flops'  # ['err&params', 'err&flops', 'err&params&flops']
benchmark = NASBench101Benchmark(objs=objs, normalized_objectives=False)
print("Benchmaking on NB101 search space with objectives: {}".format(objs))

Auto Configuration Succeed!, Using database J:\BigFiles\EvoXBench_tutorial_materials_IEEE CEC’2023 Competition on Multiobjective Neural Architecture Search\database20220713\database.
Benchmaking on NB101 search space with objectives: err&params&flops


# 2.2 How to evaluate an architecture

In [8]:
# let's randomly create N architectures
N = 1
archs = benchmark.search_space.sample(N)
print('Randomly create {} architectures:'.format(N))
print(archs)

Randomly create 1 architectures:
[{'matrix': array([[0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]]), 'ops': ['input', 'conv1x1-bn-relu', 'maxpool3x3', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'output']}]


In [9]:
# encode architecture (phenotype) to decision variables (genotypes)
X = benchmark.search_space.encode(archs)
print('Encode architectures to decision variables X: ')
print(X)

Encode architectures to decision variables X: 
[[0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 1 2 0 0 0]]


In [6]:
# Evaluate the objective values
# if true_eval is True, return mean TEST accuracy over multiple runs, 
# should only be used for final comparison.
true_eval = True
F = benchmark.evaluate(X, true_eval=true_eval)
print("Evaluating architectures for objectives: {}".format(objs))
print(F)

Evaluating architectures for objectives: err&params&flops
[[ 1. inf inf]]


In [12]:
# Evaluate the objective values
# if true_eval is False, return VALIDATION accuracy from one (randomly selected) run, 
# should be used during search
true_eval = False
print("Evaluating architectures for objectives: {}".format(objs))
for i in range(5):
    F = benchmark.evaluate(X, true_eval=true_eval)
    print("Trial {}:".format(i+1))
    print(F)

Evaluating architectures for objectives: err&params
Trial 1:
[[5.86938858e-02 4.34215400e+06]]
Trial 2:
[[6.14984035e-02 4.34215400e+06]]
Trial 3:
[[6.3100934e-02 4.3421540e+06]]
Trial 4:
[[6.3100934e-02 4.3421540e+06]]
Trial 5:
[[6.3100934e-02 4.3421540e+06]]


# 2.3 Other benchmarks

In [31]:
# NAS-Bench-201 search space
from evoxbench.benchmarks import NASBench201Benchmark
# hardware = 'edgegpu'  # ['edgegpu', 'raspi4', 'edgetpu', 'pixel3', 'eyeriss', 'fpga']
# ['err&params', 'err&flops', 'err&latency', 'err&params&flops', 'err&params&latency', ...]
objs = 'err&params&flops&edgegpu_latency&edgegpu_energy'
benchmark = NASBench201Benchmark(objs=objs, normalized_objectives=False)
print("Benchmaking on NB201 search space with objectives: {}".format(objs))
archs = benchmark.search_space.sample(1)
print(archs)
X = benchmark.search_space.encode(archs)
print(X)
F = benchmark.evaluate(X, true_eval=True)
print(F)

Benchmaking on NB201 search space with objectives: err&params&flops&edgegpu_latency&edgegpu_energy
['|avg_pool_3x3~0|+|none~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_1x1~1|avg_pool_3x3~2|']
[[4 0 3 3 2 4]]
[[ 8.40333333  0.587386   82.49409     6.83680534 31.76607656]]


In [ ]:
# none 0
# skip_connect 1
# nor_conv_1x1 2
# nor_conv_3x3 3
# avg_pool_3x3 4

In [41]:
# NATS size search space 
from evoxbench.benchmarks import NATSBenchmark
import numpy as np
objs = 'err&params&flops&latency'
# ['err&params', 'err&flops', 'err&latency', 'err&params&flops', 'err&params&latency', ...]
benchmark = NATSBenchmark(objs=objs, normalized_objectives=False)
print("Benchmaking on NATS search space with objectives: {}".format(objs))
archs = benchmark.search_space.sample(1)
print(archs)
X = benchmark.search_space.encode(archs)
print(X,np.min(X),np.max(X))
F = benchmark.evaluate(X, true_eval=True)
print(F)

Benchmaking on NATS search space with objectives: err&params&flops&latency
['40:64:8:48:16']
[[4 7 0 5 1]] 0 7
[[9.27000000e+00 1.94498000e-01 9.67067300e+01 1.55204707e-02]]


In [44]:
# DARTS search space
from evoxbench.benchmarks import DARTSBenchmark
objs = 'err&params'  # ['err&params', 'err&flops', 'err&params&flops']
benchmark = DARTSBenchmark(objs=objs, normalized_objectives=False)
print("Benchmaking on DARTS search space with objectives: {}".format(objs))
archs = benchmark.search_space.sample(10)
X = benchmark.search_space.encode(archs)
print(X)

Benchmaking on DARTS search space with objectives: err&params
[[5 0 0 1 0 0 2 2 6 2 0 1 3 2 6 0 4 1 4 0 1 1 5 2 5 2 4 0 6 1 6 3]
 [3 1 5 0 4 1 3 0 6 0 3 1 3 1 1 0 1 0 5 1 2 1 0 2 0 0 4 2 0 2 5 1]
 [2 1 4 0 1 2 0 0 0 0 4 3 6 1 3 2 4 0 5 1 0 1 3 2 1 1 2 2 1 2 6 0]
 [0 0 4 1 5 0 6 2 0 2 1 0 0 3 4 4 0 0 0 1 5 1 0 2 6 1 5 0 3 2 5 1]
 [1 0 4 1 0 0 3 1 0 3 4 2 0 1 4 3 3 1 0 0 4 0 6 1 1 2 1 0 1 0 6 3]
 [1 1 5 0 5 0 1 2 6 1 0 2 0 3 4 1 1 1 1 0 2 0 2 1 2 3 6 1 5 4 4 2]
 [4 1 3 0 1 0 2 2 4 2 6 3 3 4 6 1 2 1 3 0 2 2 0 0 1 2 6 0 1 2 6 4]
 [0 1 0 0 2 2 6 0 5 3 1 0 5 4 4 1 6 1 5 0 5 1 0 0 4 3 4 2 1 4 6 1]
 [1 1 6 0 2 1 0 0 2 0 3 3 0 0 6 3 1 0 1 1 6 0 4 1 3 1 6 0 6 4 5 0]
 [3 1 3 0 3 2 3 0 1 1 1 0 5 4 0 1 6 1 4 0 4 1 3 0 6 1 6 2 3 2 0 1]]


In [45]:
# DARTS search space
from evoxbench.benchmarks import DARTSBenchmark
objs = 'err&params'  # ['err&params', 'err&flops', 'err&params&flops']
benchmark = DARTSBenchmark(objs=objs, normalized_objectives=False)
print("Benchmaking on DARTS search space with objectives: {}".format(objs))
archs = benchmark.search_space.sample(1)
print(archs)
X = benchmark.search_space.encode(archs)
print(X)
F = benchmark.evaluate(X, true_eval=True)
print(F[0,0])

Benchmaking on DARTS search space with objectives: err&params
[Genotype(normal=[('avg_pool_3x3', 1), ('max_pool_3x3', 0), ('skip_connect', 2), ('avg_pool_3x3', 1), ('skip_connect', 3), ('sep_conv_3x3', 2), ('dil_conv_3x3', 1), ('sep_conv_5x5', 3)], normal_concat=[2, 3, 4, 5], reduce=[('sep_conv_3x3', 0), ('sep_conv_3x3', 1), ('avg_pool_3x3', 1), ('sep_conv_5x5', 0), ('sep_conv_3x3', 3), ('dil_conv_3x3', 0), ('sep_conv_5x5', 2), ('dil_conv_5x5', 3)], reduce_concat=[2, 3, 4, 5])]
[[1 1 2 0 0 2 1 1 0 3 3 2 5 1 4 3 3 0 3 1 1 1 4 0 3 3 5 0 4 2 6 3]]
0.06990354924561082


In [12]:
import os
from collections import namedtuple
from ConfigSpace.read_and_write import json as cs_json
import nasbench301 as nb

models_dir = os.path.join('/opt/data/private/BigFiles', 'nb_models_1.0')
model_paths = {
    model_name : os.path.join(models_dir, '{}_v1.0'.format(model_name))
    for model_name in ['xgb', 'gnn_gin', 'lgb_runtime']
}
print("==> Loading performance surrogate model...")
ensemble_dir_performance = model_paths['xgb']
print(ensemble_dir_performance)
performance_model = nb.load_ensemble(ensemble_dir_performance)

# Load the runtime surrogate model
#NOTE: Defaults to using the default model download path
print("==> Loading runtime surrogate model...")
ensemble_dir_runtime = model_paths['lgb_runtime']
runtime_model = nb.load_ensemble(ensemble_dir_runtime)

tensorflow is not installed.
==> Loading performance surrogate model...
/opt/data/private/BigFiles/nb_models_1.0/xgb_v1.0
[11:17:35] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[11:17:37] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[11:17:39] WARNING: ../src/l

In [21]:
# Predict
print("==> Predict runtime and performance...")
prediction_genotype = performance_model.predict(config=archs[0], representation="genotype", with_noise=False)
runtime_genotype = runtime_model.predict(config=archs[0], representation="genotype")
print("Genotype architecture performance: %f, runtime %f" %(prediction_genotype, runtime_genotype))

==> Predict runtime and performance...
Genotype architecture performance: 93.472656, runtime 5595.149626


In [5]:
# ResNet50D search space 
from evoxbench.benchmarks import ResNet50DBenchmark
objs = 'err&params&flops'
# ['err&params', 'err&flops', 'err&params&flops']
benchmark = ResNet50DBenchmark(objs=objs, normalized_objectives=False)
print("Benchmaking on ResNet50D search space with objectives: {}".format(objs))
archs = benchmark.search_space.sample(1)
print(archs)
X = benchmark.search_space.encode(archs)
print(X)
F = benchmark.evaluate(X, true_eval=True)
print(F)

Benchmaking on ResNet50D search space with objectives: err&params&flops
[{'r': 128, 'w': [1, 2, 0, 0, 0, 1], 'e': [0.25, 0.35, 0.25, 0.35, 0.35, 0.25, 0.35, 0.25, 0.25, 0.25, 0.25, 0.2, 0.25, 0.2, 0.2, 0.2, 0.35, 0.2], 'd': [0, 2, 1, 2, 2]}]
[[0 0 0 1 1 1 2 2 3 2 3 3 2 3 0 2 2 2 1 2 1 1 1 3 1]]
[[2.71153254e-01 1.69836400e+07 1.54778869e+09]]


In [6]:
# Transformer search space 
from evoxbench.benchmarks import TransformerBenchmark
objs = 'err&params&flops'
# ['err&params', 'err&flops', 'err&params&flops']
benchmark = TransformerBenchmark(objs=objs, normalized_objectives=False)
print("Benchmaking on Transformer search space with objectives: {}".format(objs))
archs = benchmark.search_space.sample(1)
print(archs)
X = benchmark.search_space.encode(archs)
print(X)
F = benchmark.evaluate(X, true_eval=True)
print(F)

Benchmaking on Transformer search space with objectives: err&params&flops
[{'depth': 15, 'embed_dim': 624, 'mlp_ratio': [4.0, 3.0, 4.0, 3.5, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.5, 3.5, 3.5, 3.5], 'num_heads': [9, 10, 10, 9, 9, 10, 10, 10, 9, 9, 9, 10, 10, 9, 10]}]
[[1 2 2 0 2 1 0 0 2 2 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 1 1 0 1 0]]
[[1.81047950e-01 6.44097280e+07 1.34198077e+10]]


In [7]:
# MobileNetV3 search space 
from evoxbench.benchmarks import MobileNetV3Benchmark
objs = 'err&params&flops'
# ['err&params', 'err&flops', 'err&latency', 'err&params&flops', 'err&params&latency', ...]
benchmark = MobileNetV3Benchmark(objs=objs, normalized_objectives=False)
print("Benchmaking on MobileNetV3 search space with objectives: {}".format(objs))
archs = benchmark.search_space.sample(1)
print(archs)
X = benchmark.search_space.encode(archs)
print(X)
F = benchmark.evaluate(X, true_eval=True)
print(F)

Benchmaking on MobileNetV3 search space with objectives: err&params&flops
[{'r': 192, 'ks': [7, 7, 5, 7, 3, 7, 3, 5, 3, 7, 5, 5, 3, 3, 5, 7, 7, 3, 3, 5], 'e': [3, 6, 6, 3, 6, 3, 6, 3, 6, 4, 4, 4, 3, 6, 3, 3, 3, 3, 4, 3], 'd': [4, 4, 3, 4, 4]}]
[[2 3 9 8 3 7 3 7 2 7 6 5 0 1 7 2 3 3 1 4 2]]
[[2.30890867e-01 6.74452000e+06 7.89135696e+08]]
